# Etude immobilière

## Import

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
import plotly.express as px


In [3]:
#Getting data straight from online url

url_dict= {
    
    '2021':'https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20220408-143516/valeursfoncieres-2021.txt',
    '2020': 'https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20220408-143240/valeursfoncieres-2020.txt',
    '2019':'https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20220408-142854/valeursfoncieres-2019.txt',
    '2018':'https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20220408-142623/valeursfoncieres-2018.txt'
    }
data=[]
for year,url in tqdm(url_dict.items()):
    df=pd.read_csv(url, sep='|')
    data.append(df)



  0%|          | 0/4 [00:00<?, ?it/s]

/Users/abdellatif/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (18,23,24,25,26,27,28,29,30,31,33,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abdellatif/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abdellatif/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abdellatif/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/IPython/core/interact

In [4]:
df = pd.concat(data)



In [5]:
df.head()

,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2021,Vente,...,NaN,0,1.0,Maison,NaN,97.0,5.0,S,NaN,2410.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2021,Vente,...,NaN,0,3.0,Dépendance,NaN,0.0,0.0,S,NaN,2410.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/01/2021,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,530.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2021,Vente,...,NaN,0,1.0,Maison,NaN,88.0,4.0,S,NaN,866.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/01/2021,Vente,...,NaN,0,1.0,Maison,NaN,168.0,5.0,S,NaN,1426.0


## Preprocessing

In [6]:

df = df.drop(['Code service CH','Reference document','1 Articles CGI','2 Articles CGI','3 Articles CGI'
        ,'4 Articles CGI','5 Articles CGI','B/T/Q','Surface Carrez du 1er lot','Surface Carrez du 2eme lot',
       'Surface Carrez du 3eme lot','Surface Carrez du 4eme lot','Surface Carrez du 5eme lot','Nature culture',
        'Nature culture speciale','Surface terrain'],axis=1)
df = df.drop(['1er lot','2eme lot','3eme lot','4eme lot','5eme lot'],axis=1)

In [7]:
df['Date mutation'] = pd.to_datetime(df['Date mutation'], format='%d/%m/%Y')
df['year'] = df['Date mutation'].dt.year
df = df.drop_duplicates()
df = df[df['Type local'].isin(['Maison','Appartement'])]
df['Valeur fonciere'] = df['Valeur fonciere'].str.replace(',','.')
df['Valeur fonciere']  = df['Valeur fonciere'].astype(float)
df['Prix M2']=df['Valeur fonciere']/df['Surface reelle bati']
df['strnovoie']=df['No voie'].astype(str)
df['strnovoie']= df['strnovoie'].str.slice(stop=2)
df['Adresse'] = df['strnovoie']+'+'+df['Voie'].str.lower()+'+'+df['Code postal'].astype(str).str[0:-2]
df['lat']=0
df['lat']= df['lat'].astype(float)
df['lon']=0
df['lon']= df['lon'].astype(float)
    

## LH

In [8]:
dlh=df[df['Code postal'].isin([76600,76610,76620])].copy()


In [9]:
dlh = dlh[dlh['Prix M2']<8000]

In [10]:
fig = px.box(dlh,x='year',y='Prix M2',hover_data=['No voie','Voie','Code postal','Prix M2'])
fig.show()

In [13]:
dlh.groupby(['Type local','Code postal','year']).mean()

No disposition  Valeur fonciere     No voie  \
Type local  Code postal year                                                
Appartement 76600.0     2018        1.009787    129396.236834   76.090017   
                        2019        1.008503    135208.741039   84.098002   
                        2020        1.000542    142721.276035   69.684067   
                        2021        1.003153    141851.221477  153.008408   
            76610.0     2018        1.000000     72334.666667   74.346667   
                        2019        1.000000     69713.050000  106.325000   
                        2020        1.047059    104331.235294   95.964706   
                        2021        1.000000     97716.010417   60.687500   
            76620.0     2018        1.012295    130076.684426  109.004098   
                        2019        1.000000    125736.503759   74.553360   
                        2020        1.000000    124126.533333   66.677778   
                        2021        1.003906    134030.199219   71.726562   
Maison      76600.0     2018        1.000000    185840.871447   79.893617   
                        2019        1.000000    196077.189602   69.088685   
                        2020        1.004098    198492.008197   45.110656   
                        2021        1.010830    231616.891913   49.010830   
            76610.0     2018        1.017937    157116.128430   45.434978   
                        2019        1.000000    155152.497942   47.440329   
                        2020        1.000000    174682.880184  208.861751   
                        2021        1.012987    180656.536797   87.458874   
            76620.0     2018        1.002024    175471.376518   38.315789   
                        2019        1.001957    184492.758004   73.825832   
                        2020        1.002049    195284.737152   42.186475   
                        2021        1.008214    205822.034209   36.162218   

                              Code commune  Prefixe de section     No plan  \
Type local  Code postal year                                                 
Appartement 76600.0     2018    351.000000                 NaN  139.683938   
                        2019    351.000000                 NaN  139.216344   
                        2020    351.000000                 NaN  208.165222   
                        2021    351.000000                 NaN  170.924855   
            76610.0     2018    351.000000                 NaN  113.546667   
                        2019    351.000000               539.0  130.450000   
                        2020    351.000000               539.0  132.411765   
                        2021    351.000000               539.0  129.447917   
            76620.0     2018    351.000000                 NaN  171.213115   
                        2019    351.000000                 NaN  230.289474   
                        2020    351.000000                 NaN  179.077778   
                        2021    351.000000                 NaN  179.730469   
Maison      76600.0     2018    351.804255               539.0  293.519149   
                        2019    351.577982                 NaN  264.782875   
                        2020    351.000000                 NaN  327.405738   
                        2021    351.000000                 NaN  276.050542   
            76610.0     2018    351.000000               539.0  143.430493   
                        2019    351.000000               539.0  139.588477   
                        2020    351.000000               539.0  146.599078   
                        2021    351.000000               539.0  134.333333   
            76620.0     2018    351.000000                 NaN  206.085020   
                        2019    351.000000                 NaN  190.318982   
                        2020    351.000000                 NaN  190.196721   
                        2021    351.000000                 NaN  187.061602   

In [11]:
fig = px.line(dlh[dlh['Voie']=='ETIENNE MEHUL'].sort_values('Date mutation'), x="Date mutation", y="Prix M2", title='Evolution du prix M2',hover_data=['No voie','Voie','Code postal','Prix M2','Valeur fonciere','Surface reelle bati'])
fig.show()

## Modèle Stat

In [12]:

y = dlh['Prix M2']
X= dlh[['Nombre de lots','year','Nombre pieces principales','Surface reelle bati','Code postal']]


In [28]:
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector

ohe = OneHotEncoder()

pipe = make_column_transformer((ohe,['Code postal']))





,Nombre de lots,year,Nombre pieces principales,Surface reelle bati,Code postal
2454617,1,2021,2.0,39.0,76600.0
2454620,0,2021,4.0,100.0,76600.0
2454621,2,2021,1.0,32.0,76620.0
2454623,0,2021,7.0,106.0,76620.0
2454624,0,2021,4.0,95.0,76620.0
...,...,...,...,...,...
2498983,0,2018,2.0,35.0,76620.0
2498990,0,2018,1.0,23.0,76620.0
2498994,2,2018,2.0,42.0,76600.0
2498995,1,2018,2.0,43.0,76600.0


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [33]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor

regr = LinearRegression()

In [34]:
regr.fit(X_train,y_train)

LinearRegression()

In [35]:
pipeline = make_pipeline(pipe,regr)

In [43]:
from sklearn.model_selection import cross_val_score, cross_validate
score_pipe_ensemble_cv=cross_validate(pipeline, X, y, cv = 5)

In [45]:
score_pipe_ensemble_cv

{'fit_time': array([0.015728  , 0.01091313, 0.00608277, 0.00639892, 0.00646615]),
 'score_time': array([0.00247407, 0.00149298, 0.00157905, 0.00173807, 0.0017519 ]),
 'test_score': array([ 0.0067915 ,  0.00994888,  0.01765809,  0.02334067, -0.01300558])}

In [20]:
for name, feature in zip(['Nombre de lots','year','Nombre pieces principales','Surface reelle bati','Code postal'], regr.feature_importances_):
    print(name, round(feature*100,1))

Nombre de lots 38.8
year 4.9
Nombre pieces principales 2.2
Surface reelle bati 34.9
Code postal 19.2


In [59]:
y_pred = regr.predict(X_test)*105


In [57]:
X_maison = pd.DataFrame({'Nombre de lots':[2],'year':[2021],'Nombre pieces principales':[5],'Surface reelle bati':[105],'Code postal':[76620]})

#['Nombre de lots','year','Nombre pieces principales','Surface reelle bati','Code postal']

In [60]:
regr.predict(X_maison)*105

array([177998.30949637])

In [22]:
y_pred

array([2203.3860782 , 5334.8038466 , 2185.75634145, ..., 2524.46563427,
       2183.4194805 , 5055.72981975])

In [23]:
comparaison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred,'Différence':y_pred-y_test})
comparaison.mean()

Actual        2338.214181
Predicted     2346.313974
Différence       8.099793
dtype: float64

## API 

In [47]:
import requests

requests.get('https://api-adresse.data.gouv.fr/search/?q=8+bd+du+port').json()

{'type': 'FeatureCollection',
 'version': 'draft',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [2.290084, 49.897442]},
   'properties': {'label': '8 Boulevard du Port 80000 Amiens',
    'score': 0.492192009569378,
    'housenumber': '8',
    'id': '80021_6590_00008',
    'name': '8 Boulevard du Port',
    'postcode': '80000',
    'citycode': '80021',
    'x': 648952.58,
    'y': 6977867.14,
    'city': 'Amiens',
    'context': '80, Somme, Hauts-de-France',
    'type': 'housenumber',
    'importance': 0.67727,
    'street': 'Boulevard du Port'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [2.062794, 49.0317]},
   'properties': {'label': '8 Boulevard du Port 95000 Cergy',
    'score': 0.49167382775119606,
    'housenumber': '8',
    'id': '95127_1448_00008',
    'name': '8 Boulevard du Port',
    'postcode': '95000',
    'citycode': '95127',
    'x': 631466.41,
    'y': 6881718.82,
    'city': 'Cergy',
    'context': "95, V

## API CSV

In [48]:
def get_ln(adresse):
    url= f'https://api-adresse.data.gouv.fr/search/?q={adresse}'
    lon= None
    r = requests.get(url)
    if r.status_code !=200:
        return None
    else :
        lon = r.json()['features'][0]['geometry']['coordinates'][0]
       
    return lon

def get_lt(adresse):
    url= f'https://api-adresse.data.gouv.fr/search/?q={adresse}'
    lat= None
    r = requests.get(url)
    if r.status_code !=200:
        return None
    else :
        
        lat = r.json()['features'][0]['geometry']['coordinates'][1]
        
    return lat

In [49]:
def get_ltln(adresse):
    url= f'https://api-adresse.data.gouv.fr/search/?q={adresse}'
    lat= None
    lon= None
    r = requests.get(url)
    rr=r.json()
    if r.status_code !=200:
        return None, None
    else :
        try:
            
            lat = rr['features'][0]['geometry']['coordinates'][1]
            lon = rr['features'][0]['geometry']['coordinates'][0]
        except:
            pass
        
    return lat, lon

In [50]:
def add_values(row):
    col='Adresse'
    advalue=row[col]
    adress_lat, adress_lon= get_ltln(advalue)
    #adress_lat= get_ln(advalue)
    row['lat']=adress_lat
    row['lon']=adress_lon
    return row


In [51]:
get_ltln('51+michelet+76600')

(49.495703, 0.120818)

In [52]:
dlh = dlh.apply(add_values,axis=1)

In [54]:
dlh

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,year,Prix M2,strnovoie,Adresse,lat,lon
2454617,1,2021-01-08,Vente,59200.0,51.0,RUE,5695,MICHELET,76600.0,LE HAVRE,...,Appartement,NaN,39.0,2.0,2021,1517.948718,51,51+michelet+76600,49.495703,0.120818
2454620,1,2021-01-07,Vente,365000.0,16.0,PAS,0400,ARCADE NOURY,76600.0,LE HAVRE,...,Maison,NaN,100.0,4.0,2021,3650.000000,16,16+arcade noury+76600,49.498122,0.107232
2454621,1,2021-01-06,Vente,76000.0,13.0,RUE,4805,LECAT,76620.0,LE HAVRE,...,Appartement,NaN,32.0,1.0,2021,2375.000000,13,13+lecat+76620,49.502667,0.116413
2454623,1,2021-01-08,Vente,185000.0,25.0,RUE,1405,DU CHATEAU DE CARENCY,76620.0,LE HAVRE,...,Maison,NaN,106.0,7.0,2021,1745.283019,25,25+du chateau de carency+76620,49.508577,0.132067
2454624,1,2021-01-07,Vente,252000.0,22.0,RUE,5565,MAURICE COSNIER,76620.0,LE HAVRE,...,Maison,NaN,95.0,4.0,2021,2652.631579,22,22+maurice cosnier+76620,49.509320,0.103451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498983,1,2018-06-28,Adjudication,100200.0,41.0,RUE,4455,JEANNETTE,76620.0,LE HAVRE,...,Appartement,NaN,35.0,2.0,2018,2862.857143,41,41+jeannette+76620,49.515109,0.126075
2498990,1,2018-06-28,Adjudication,100200.0,41.0,RUE,4455,JEANNETTE,76620.0,LE HAVRE,...,Appartement,NaN,23.0,1.0,2018,4356.521739,41,41+jeannette+76620,49.515109,0.126075
2498994,1,2018-03-26,Vente,60000.0,91.0,RUE,6955,RICHELIEU,76600.0,LE HAVRE,...,Appartement,NaN,42.0,2.0,2018,1428.571429,91,91+richelieu+76600,49.489308,0.106738
2498995,1,2018-03-26,Vente,1.0,11.0,RUE,1065,DE BRETAGNE,76600.0,LE HAVRE,...,Appartement,NaN,43.0,2.0,2018,0.023256,11,11+de bretagne+76600,49.489255,0.111706


In [23]:
import plotly.express as px


fig = px.scatter_mapbox(dlh, lat="lat", lon="lon", color="Prix M2", hover_data= ['Valeur fonciere','Voie','Surface reelle bati'],
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15,mapbox_style="carto-positron")
fig.show()

In [61]:
import re
import joblib
joblib.dump(regr, 'model/model.joblib')

['model/model.joblib']